In [1]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Lecture 11 #

## Grouping by Two Columns ##

In [2]:
survey = Table.read_table('welcome_survey_v3.csv')

In [3]:
survey.group(['Handedness','Sleep Side']).show()

Handedness,Sleep Side,count
Ambidextrous,back,3
Ambidextrous,left side,2
Ambidextrous,right side,1
Ambidextrous,stomach,2
Left-handed,back,29
Left-handed,left side,25
Left-handed,right side,30
Left-handed,stomach,19
Right-handed,back,275
Right-handed,left side,285


## Pivot Tables ##

In [4]:
survey.pivot('Sleep Side', 'Handedness')

Handedness,back,left side,right side,stomach
Ambidextrous,3,2,1,2
Left-handed,29,25,30,19
Right-handed,275,285,380,193


In [5]:
survey.pivot('Sleep Side', 'Handedness', values='Extraversion', collect=np.average)

Handedness,back,left side,right side,stomach
Ambidextrous,3,3.5,3,4.5
Left-handed,5.75862,6.12,5.86667,6.31579
Right-handed,5.76364,5.71579,5.83684,5.72021


In [6]:
survey.group('Handedness', np.average)

Handedness,Year average,Extraversion average,Number of Texters average,Hours of Sleep average,Sleep Side average
Ambidextrous,,3.5,6.25,6.375,
Left-handed,,5.98058,6.93204,7.10194,
Right-handed,,5.76876,7.62136,7.01518,


## Challenge Question ##

In [7]:
nba = Table.read_table('nba_salaries.csv').relabeled(3, 'SALARY')
nba

PLAYER,POSITION,TEAM,SALARY
Paul Millsap,PF,Atlanta Hawks,18.6717
Al Horford,C,Atlanta Hawks,12
Tiago Splitter,C,Atlanta Hawks,9.75625
Jeff Teague,PG,Atlanta Hawks,8
Kyle Korver,SG,Atlanta Hawks,5.74648
Thabo Sefolosha,SF,Atlanta Hawks,4
Mike Scott,PF,Atlanta Hawks,3.33333
Kent Bazemore,SF,Atlanta Hawks,2
Dennis Schroder,PG,Atlanta Hawks,1.7634
Tim Hardaway Jr.,SG,Atlanta Hawks,1.30452


In [8]:
starters_g = nba.drop('PLAYER').group(['POSITION', 'TEAM'], max)
starters_g

POSITION,TEAM,SALARY max
C,Atlanta Hawks,12
C,Boston Celtics,2.61698
C,Brooklyn Nets,1.3629
C,Charlotte Hornets,13.5
C,Chicago Bulls,13.4
C,Cleveland Cavaliers,14.2609
C,Dallas Mavericks,5.2
C,Denver Nuggets,5.6135
C,Detroit Pistons,6.5
C,Golden State Warriors,13.8


In [9]:
starters_p = nba.pivot('POSITION','TEAM', values='SALARY', collect=max)

In [10]:
starters_p.with_column('TOTAL', starters_p.drop('TEAM').apply(sum))

TEAM,C,PF,PG,SF,SG,TOTAL
Atlanta Hawks,12,18.6717,8,4,5.74648,48.4181
Boston Celtics,2.61698,5,7.73034,6.79612,3.42551,25.5689
Brooklyn Nets,1.3629,11.236,6.3,24.8949,3.42551,47.2192
Charlotte Hornets,13.5,7,12,6.3314,13.1253,51.9567
Chicago Bulls,13.4,5.54373,20.0931,2.38044,16.4075,57.8247
Cleveland Cavaliers,14.2609,19.689,16.4075,22.9705,8.98876,82.3166
Dallas Mavericks,5.2,15.4937,5.37897,15.3615,1.449,42.8832
Denver Nuggets,5.6135,11.236,4.345,14,1.58448,36.7789
Detroit Pistons,6.5,0,13.913,2.84196,6.27,29.525
Golden State Warriors,13.8,14.2609,11.3708,11.7105,15.501,66.6431


## Joins ##

In [11]:
drinks = Table(['Drink', 'Cafe', 'Price']).with_rows([
    ['Milk Tea', 'Tea One', 4],
    ['Espresso', 'Nefeli',  2],
    ['Latte',    'Nefeli',  3],
    ['Espresso', "Abe's",   2]
])
drinks

Drink,Cafe,Price
Milk Tea,Tea One,4
Espresso,Nefeli,2
Latte,Nefeli,3
Espresso,Abe's,2


In [12]:
discounts = Table().with_columns(
    'Coupon % off', make_array(25, 50, 5),
    'Location', make_array('Tea One', 'Nefeli', 'Tea One')
)
discounts

Coupon % off,Location
25,Tea One
50,Nefeli
5,Tea One


In [13]:
combined = drinks.join('Cafe', discounts, 'Location')
combined

Cafe,Drink,Price,Coupon % off
Nefeli,Espresso,2,50
Nefeli,Latte,3,50
Tea One,Milk Tea,4,25
Tea One,Milk Tea,4,5


In [14]:
combined.with_column('Discounted Price', 
                    combined.column('Price') * (1 - combined.column('Coupon % off') / 100))

Cafe,Drink,Price,Coupon % off,Discounted Price
Nefeli,Espresso,2,50,1
Nefeli,Latte,3,50,1.5
Tea One,Milk Tea,4,25,3
Tea One,Milk Tea,4,5,3.8


In [15]:
drinks.join('Cafe', drinks)

Cafe,Drink,Price,Drink_2,Price_2
Abe's,Espresso,2,Espresso,2
Nefeli,Espresso,2,Espresso,2
Nefeli,Espresso,2,Latte,3
Nefeli,Latte,3,Espresso,2
Nefeli,Latte,3,Latte,3
Tea One,Milk Tea,4,Milk Tea,4


## Maps

In [16]:
stations = Table.read_table('station.csv')
stations

station_id,name,lat,long,dockcount,landmark,installation
2,San Jose Diridon Caltrain Station,37.3297,-121.902,27,San Jose,8/6/2013
3,San Jose Civic Center,37.3307,-121.889,15,San Jose,8/5/2013
4,Santa Clara at Almaden,37.334,-121.895,11,San Jose,8/6/2013
5,Adobe on Almaden,37.3314,-121.893,19,San Jose,8/5/2013
6,San Pedro Square,37.3367,-121.894,15,San Jose,8/7/2013
7,Paseo de San Antonio,37.3338,-121.887,15,San Jose,8/7/2013
8,San Salvador at 1st,37.3302,-121.886,15,San Jose,8/5/2013
9,Japantown,37.3487,-121.895,15,San Jose,8/5/2013
10,San Jose City Hall,37.3374,-121.887,15,San Jose,8/6/2013
11,MLK Library,37.3359,-121.886,19,San Jose,8/6/2013


In [17]:
Marker.map_table(stations.select('lat', 'long', 'name'))

In [18]:
sf = stations.where('landmark', 'San Francisco')
Circle.map_table(sf.select('lat', 'long', 'name'), color='green', radius=10)